# Credit Risk Resampling Techniques

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [5]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


* We have two columns in this source DataFrame are not numeric. Homeowner and loan_status.
* We are definitely going to want to transform these values to numeric using Integer Encoding? 
* The positive event in this data is predicting high_risk for loans. low_risk should be 0, high_risk should be 1. 

# Split the Data into Training and Testing

In [6]:
# Create our features
# Our features columns are everything EXCEPT the loan_status column

X = df.copy()                                                                         # make a copy of df
X.drop(columns='loan_status', inplace=True)                                           # simply drop the loan_status column, since that is the y-target we're looking for in the model. 
X = pd.get_dummies(X)

# Create our target
# In this case our taget is the 'loan status' column 
y = df['loan_status'] 

In [7]:
X.describe()     # we should see every column except loan_status. The homeowner column is not included because it's not numeric. 

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [18]:
# Check the balance of our target values
y['loan_status'].value_counts()                                 # Code fails??? Don't know what this error is about, but this should not influence on our models running.  
                                                                # Our target y in previous cell is already formatted and shaped correctly. 

KeyError: 'loan_status'

In [19]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [20]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
# YOUR CODE HERE
scaler = StandardScaler()

In [21]:
type(scaler)

sklearn.preprocessing._data.StandardScaler

**REMEMBER:** Standard Scaler helps to get standardized distribution, with a zero mean and standard deviation of one (unit variance). It works only on numeric values for obvious reasons, Convert the string/text columns to numerical values using one of the encoding techniques (i.e. hot/lget_dummies or label encoding) before using the standard scaler.

In [22]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [23]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [24]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [25]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9520479254722232

In [26]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  563,    56],
       [  102, 18663]], dtype=int64)

In [17]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.91      0.99      0.88      0.95      0.90       619
   low_risk       1.00      0.99      0.91      1.00      0.95      0.91     18765

avg / total       0.99      0.99      0.91      0.99      0.95      0.91     19384



**ANALYSIS:**
* Of all acutal high_risk, we were able to classify 99% correctly, or 18,663 True Positves out of 18,765 actual high_risk applicants
* 102 False Negatives, high_risk applicants that were 
* Recall is 91% which is acceptable. We captured a lot of high_risk applicants.

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [27]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE

from imblearn.over_sampling import RandomOverSampler                     # import the RandomOverSampler from imblearn 

ros = RandomOverSampler(random_state=1)                                  # create random oversampling (ros) seed so we can resample our training data.

X_resampled, y_resampled = ros.fit_resample(X_train, y_train)            # resample the training data by fitting X_train and y_train into our oversampling seed.

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled)                                                     # this should display an equal number of datapoints for both classes, by increasing 

Counter({'low_risk': 56271, 'high_risk': 56271})

In [28]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_ros = LogisticRegression(solver='lbfgs', random_state=1)            # define a new model variable for our random oversampling 
model_ros.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [29]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred_ros = model_ros.predict(X_test)                                    # make new predictions based on random oversampling 
balanced_accuracy_score(y_test, y_pred_ros)                               # generate out 

0.9936781215845847

In [30]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred_ros)                                       # making sure to pass y-predictions on the random oversampling model

array([[  615,     4],
       [  116, 18649]], dtype=int64)

In [31]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_ros))                # report based on model_ros

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



**ANALYSIS:**
* Model peforms a little worse when predicting high_risk applicants. Despute balanced accuracy score increase from  95% to 99%
* False Negatives increased from 102 to 116
* True Positves went down as well. 
* Changes are marginal, but we missed categorizing high_risk applicants. 
* We have gotten better at not wrongly classifying low_risk applicants.

### SMOTE Oversampling

In [32]:
# Resample the training data with SMOTE
# YOUR CODE HERE
from imblearn.over_sampling import SMOTE

X_resampled_smte, y_resampled_smte =SMOTE(
    random_state=1,                                # pass random_state
    sampling_strategy=1.0                          # sampling_strategy=1.0, is # you pass that corresponds to number of samples you'd like. pass 1.0 to balance evenly between gold and purple
).fit_resample(X_train, y_train)

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled_smte)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [37]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_smte = LogisticRegression(solver='lbfgs', random_state=1)
model_smte.fit(X_resampled_smte, y_resampled_smte)

LogisticRegression(random_state=1)

In [38]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred_smte = model_smte.predict(X_test)
balanced_accuracy_score(y_test, y_pred_smte)

0.9936781215845847

In [39]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred_smte)

array([[  615,     4],
       [  116, 18649]], dtype=int64)

In [40]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_smte))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [41]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE
from imblearn.under_sampling import RandomUnderSampler                           # Import random under sampler

rus = RandomUnderSampler(random_state=1)

X_resampled_rus, y_resampled_rus = rus.fit_resample(X_train, y_train)

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled_rus)

Counter({'high_risk': 1881, 'low_risk': 1881})

In [42]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_rus = LogisticRegression(solver='lbfgs', random_state=1)                    # define model based on random undersampling method
model_rus.fit(X_resampled_rus, y_resampled_rus)                                   # fit model with resampled training data from RandomUnderSampler

LogisticRegression(random_state=1)

In [43]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
y_pred_rus = model_smte.predict(X_test)                                            # make predictions based on random undersampling 
balanced_accuracy_score(y_test, y_pred_rus)

0.9936781215845847

In [44]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred_rus)

array([[  615,     4],
       [  116, 18649]], dtype=int64)

In [45]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_rus))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [46]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE
from imblearn.combine import SMOTEENN                                                        # Import SMOTEENN

smote_enn = SMOTEENN(random_state=1)                                                         # Build the SMOTEENN object
X_resampled_smtn, y_resampled_smtn = smote_enn.fit_resample(X_train, y_train)                # resample the training data with the SMOTEENN objecy

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled_smtn)

Counter({'high_risk': 55626, 'low_risk': 55948})

In [47]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_smtn = LogisticRegression(solver='lbfgs', random_state=1)
model_smtn.fit(X_resampled_smtn, y_resampled_smtn)

LogisticRegression(random_state=1)

In [48]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
y_pred_smtn = model_smte.predict(X_test)                                                     # predictions based on SMOTEENN model
balanced_accuracy_score(y_test, y_pred_smtn)                                                  

0.9936781215845847

In [49]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred_smtn)

array([[  615,     4],
       [  116, 18649]], dtype=int64)

In [50]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_smtn))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   YOUR ANSWER HERE.
   
   **Looking at all the models, it seems any model that Logisitc Regression fared the same with 99.3% balanced accuracy.**
   
   * Logistic Regression score = 95.2%
   * Random Oversample score = 99.3%
   * SMOTE score = 99.3%
   * Undersampling score = 99.3%
   * Combo Sampling score = 99.3%

2. Which model had the best recall score?

    YOUR ANSWER HERE.
    
    **Logisitc Regression had the lower recall at 91%. All other models were 99%. But I'm not understanding how that is considering looking at all the confusion matrices for those models,
    we had increase False Negatives in the higher recall models. The Logisitc Regression had fewer False Negatives**
    

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.
    
    **All models except Logistic Regression had a better score at 99%**
